<a href="https://colab.research.google.com/github/poltorashka-s-BMa/course-os-linux/blob/main/E_Papers_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задача F: Классификация научных статей по цитированиям
Добро пожаловать на соревнование!

В этом ноутбуке мы разберем задачу классификации научных статей, подготовим данные, создадим простой бейзлайн и основной шаблон для построения графовой нейронной сети, а также сформируем файл с решением для отправки в систему.

### Легенда задачи

Вы работаете в научном издательстве, которое управляет огромной базой данных статей. Каждая статья относится к одной из 20 научных областей (например, "биоинформатика", "машинное обучение", "квантовая физика"). Ваша задача — восстановить утерянные метки областей для 440 статей, используя текстовые эмбеддинги и граф цитирований.

## Шаг 1: Загрузка и подготовка данных
Сначала импортируем все необходимые библиотеки и загрузим наши данные. Для этого соревнования мы сгенерируем данные прямо в ноутбуке, чтобы он был полностью самодостаточным.

In [ ]:
# Эти библиотеки вам наверняка пригодятся
import json
import random
import numpy as np
from collections import defaultdict, Counter

# Эти будут полезны для создания моделей
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv

# А эти могут понядобиться для расчета и визуализации результата
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [ ]:
# Загрузка данных
with open('embeddings_data.json', 'r') as f:
    embeddings_data = json.load(f)
with open('graph_data.json', 'r') as f:
    graph_data = json.load(f)

# Посмотрим на структуру
print("Пример элемента из embeddings_data (только ключи):")
print(embeddings_data[0].keys())
print("\nПример ребра из graph_data:")
print(graph_data[0])

# Разделим данные на обучающую и тестовую выборки
train_data = [d for d in embeddings_data if d['split'] == 'train']
test_data = [d for d in embeddings_data if d['split'] == 'test']

print(f"\nВсего вершин: {len(embeddings_data)}")
print(f"Из них в обучении (train): {len(train_data)}")
print(f"Из них в тесте (test): {len(test_data)}")
print(f"Всего ребер в графе: {len(graph_data)}")

## Шаг 2: Бейзлайн на основе структуры графа
Давайте построим простое базовое решение (бейзлайн), которое использует только структуру графа и полностью игнорирует эмбеддинги. Логика простая: для каждой тестовой вершины предскажем тот класс, который чаще всего встречается среди её соседей из обучающей выборки.

Ваша задача — заполнить пропущенные части в коде ниже.

In [ ]:
def graph_baseline_predictor(graph_data, embeddings_data, num_classes=20):
    """
    Предсказывает классы для тестовых вершин на основе самого популярного класса
    среди соседей из обучающей выборки.
    """
    adj_list = defaultdict(list)
    for u, v in graph_data:
        adj_list[u].append(v)

    node_info = {d['id']: d for d in embeddings_data}
    test_node_ids = [d['id'] for d in embeddings_data if d.get('split') == 'test']

    predictions = {}

    for node_id in test_node_ids:
        all_neighbors = adj_list.get(node_id, [])
        train_neighbors = ... # [!] ВАШ КОД: отфильтруйте соседей, оставив только те, что в 'train'

        if not train_neighbors:
            predictions[node_id] = random.randint(0, num_classes - 1)
            continue

        neighbor_labels = [node_info[n_id]['label'] for n_id in train_neighbors]

        if not neighbor_labels:
            # На случай, если соседи есть, но у них нет меток (хотя по логике выше это невозможно)
            predictions[node_id] = random.randint(0, num_classes-1)
            continue

        label_counts = Counter(neighbor_labels)
        predicted_class = ... # [!] ВАШ КОД: найдите самый частый класс
        predictions[node_id] = predicted_class

    return predictions

# --- Запуск бейзлайна ---
baseline_predictions = graph_baseline_predictor(graph_data, embeddings_data)

**Подсказка:** Если вы все сделаете правильно, точность этого бейзлайна должна быть в районе 0.4. По формуле оценки это даст вам около 2 баллов. Ваша цель — значительно превзойти этот результат!

### Шаг 3: Функция для сохранения решения
Определим функцию для выгрузки решения сразу. Она понадобится нам и для бейзлайна, и для финальной модели.

In [ ]:
def save_predictions_to_csv(predictions_dict, filename="predictions.csv"):
    """
    Сохраняет предсказания из словаря в CSV файл для отправки в систему.
    """
    # Убедимся, что predictions_dict содержит предсказания для всех тестовых вершин
    test_ids = {d['id'] for d in embeddings_data if d['split'] == 'test'}
    if set(predictions_dict.keys()) != test_ids:
        print("Внимание! В словаре предсказаний отсутствуют некоторые тестовые ID.")

    pred_df = pd.DataFrame(list(predictions_dict.items()), columns=['id', 'label'])
    pred_df = pred_df.sort_values(by='id')
    pred_df.to_csv(filename, header=False, index=False)

    print(f"\nРешение успешно сохранено в файл: {filename}")
    print("Пример содержимого файла:")
    # Магическая команда для вывода первых строк файла в Jupyter
    !head -n 5 {filename}

In [ ]:
save_predictions_to_csv(baseline_predictions, "baseline_predictions.csv")

## Шаг 4: Построение финальной модели (GNN)
Теперь ваша очередь построить графовую нейросеть! Вам предстоит пройти все этапы: подготовка данных, определение архитектуры, обучение и получение предсказаний.

### 4.1: Подготовка данных и создание выборок
Сначала подготовим данные в формате PyTorch Geometric. Самое важное — мы разделим обучающую выборку (train) на две части: новую, уменьшенную обучающую (new_train) и валидационную (val).

Валидационная выборка критически важна для:

- Настройки гиперпараметров (скорость обучения, глубина сети и т.д.).
- Отслеживания переобучения и применения Early Stopping.

In [ ]:
# --- Создание тензоров ---
all_labels = {d['id']: d.get('label', -1) for d in embeddings_data}
x = torch.tensor([d['embedding'] for d in embeddings_data], dtype=torch.float)
y = torch.tensor([all_labels[i] for i in range(len(embeddings_data))], dtype=torch.long)
edge_index = torch.tensor(graph_data, dtype=torch.long).t().contiguous()

# --- Создание масок ---
# Сначала найдем ID всех обучающих вершин
train_ids = [d['id'] for d in embeddings_data if d['split'] == 'train']

# [!] ВАШ КОД: разделите `train_ids` на `train_ids_new` и `val_ids`
# в соотношении 80/20. Используйте `train_test_split` из sklearn.
# Не забудьте про `random_state` для воспроизводимости.
train_ids_new, val_ids = ...

# Теперь создаем маски PyTorch на основе этих ID
train_mask = torch.zeros(len(embeddings_data), dtype=torch.bool)
val_mask = torch.zeros(len(embeddings_data), dtype=torch.bool)
test_mask = torch.zeros(len(embeddings_data), dtype=torch.bool)

train_mask[train_ids_new] = True
val_mask[val_ids] = True
test_mask[[d['id'] for d in embeddings_data if d['split'] == 'test']] = True

# Собираем все в один объект Data
pyg_data = Data(x=x, edge_index=edge_index, y=y,
                train_mask=train_mask, val_mask=val_mask, test_mask=test_mask)

print("Объект данных PyTorch Geometric:")
print(pyg_data)
print(f"Новый размер train: {pyg_data.train_mask.sum().item()}")
print(f"Размер validation: {pyg_data.val_mask.sum().item()}")
print(f"Размер test: {pyg_data.test_mask.sum().item()}")

### 4.2: Определение архитектуры модели
Ваша задача — спроектировать и написать класс для GNN модели.

Примерный план архитектуры:

1. Вход: Принимает объект pyg_data.
2. Слой 1: Графовый сверточный слой (например, GCNConv или GATConv), который преобразует входные эмбеддинги в промежуточное представление. После него — функция активации (например, ReLU).
3. Регуляризация: Слой Dropout для борьбы с переобучением.
4. Слой 2: Еще один графовый сверточный слой.
5. Классификатор (MLP): После графовых слоев можно добавить небольшой полносвязный слой (nn.Linear) для финальной классификации.
6. Выход: Логиты для каждого из 20 классов.

In [ ]:
class MyGNN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        # [!] ВАШ КОД: определите слои вашей модели здесь
        # self.conv1 = ...
        # self.dropout = ...
        # self.conv2 = ...
        # self.classifier_lin = ...

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # [!] ВАШ КОД: опишите прямой проход данных через слои
        # Не забудьте про функции активации и dropout

        # x = self.conv1(...)
        # x = F.relu(x)
        # x = self.dropout(x)
        # ...

        return x

### 4.3: Обучение модели
Напишите цикл обучения. Следите за loss и accuracy на обучающей и валидационной выборках.

In [ ]:
# --- Настройка ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# [!] ВАШ КОД: создайте экземпляр вашей модели, оптимизатор и функцию потерь
# model = MyGNN(...)
# optimizer = ...
# criterion = ...

pyg_data = pyg_data.to(device)

# --- Цикл обучения ---
for epoch in range(1, 301): # Можете менять число эпох
    # [!] ВАШ КОД: Напишите шаг обучения
    # 1. Переключите модель в режим обучения: model.train()
    # 2. Обнулите градиенты: optimizer.zero_grad()
    # 3. Сделайте предсказание: out = model(pyg_data)
    # 4. Посчитайте loss ТОЛЬКО на обучающей маске: loss = criterion(out[...], pyg_data.y[...])
    # 5. Сделайте шаг назад: loss.backward()
    # 6. Обновите веса: optimizer.step()

    # --- Валидация ---
    if epoch % 10 == 0:
        model.eval()
        with torch.no_grad():
            out = model(pyg_data)
            pred = out.argmax(dim=1)

            # [!] ВАШ КОД: посчитайте точность на train и val выборках
            # train_acc = ...
            # val_acc = ...

            print(f'Эпоха: {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}')

## Шаг 5: Выгрузка решения
После того, как вы обучили вашу лучшую модель и получили словарь final_predictions, его нужно сохранить в predictions.csv в правильном формате для отправки в систему.

In [ ]:
# [!] ВАШ КОД:
# 1. Переключите модель в режим оценки: model.eval()
# 2. Сделайте предсказания для всех данных
# 3. Выберите предсказания только для тестовой маски
# 4. Преобразуйте их в словарь {id: label} и сохраните в `final_predictions`

final_predictions = {} # Заполните этот словарь

# Сохраняем финальное предсказание
save_predictions_to_csv(final_predictions)

Готово! Теперь у вас есть файл predictions.csv, который можно загружать в систему. Попробуйте улучшить свой результат!